# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
#import and read the weather csv
weather_vacation = "weather_data_cities.csv"

weather_vacation_data = pd.read_csv(weather_vacation)

weather_vacation_df = pd.DataFrame(weather_vacation_data)

weather_vacation_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Ahipara,-35.1667,173.1667,65.35,72,96,11.95,NZ,1650839488
1,1,Saint-Philippe,-21.3585,55.7679,72.99,79,90,14.03,RE,1650839489
2,2,Jamestown,42.0970,-79.2353,80.24,31,0,12.66,US,1650839489
3,3,Rikitea,-23.1203,-134.9692,73.02,89,100,16.44,PF,1650839490
4,4,Udachny,66.4167,112.4000,10.17,93,58,2.48,RU,1650839490


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
# Configure gmaps
gmaps.configure(api_key=g_key)


In [23]:
# Bring in location data pairs, humidity data, define map and layer, and add layer to map

locations = weather_vacation_df[["Lat", "Lng"]].astype(float)

humidity_rate = weather_vacation_df["Humidity"].astype(float)

#create heatmap layer

figure = gmaps.figure(center=(35.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate, dissipating=False, max_intensity=100, point_radius=1)

figure.add_layer(heat_layer)

plt.savefig("humidityheatmap.png")

figure


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [35]:
#set paramaters for vacation spots

paramater1 = weather_vacation_df.loc[weather_vacation_df["Max Temp"]<90,:]
paramater2 = paramater1.loc[paramater1["Max Temp"]>70,:]
paramater3 = paramater2.loc[paramater2["Cloudiness"]==0,:]

paramater3.count()





Unnamed: 0    38
City          38
Lat           38
Lng           38
Max Temp      38
Humidity      38
Cloudiness    38
Wind Speed    38
Country       38
Date          38
dtype: int64

### Hotel Map
* Create a dataframe named `hotel_df` that is the df created above only with the pertinent data.  See the prepared code below (`info_box_template`) to determine what data you will need for your final map.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels (aka "lodging") within 5000 meters.
* Utilize the Google Places API to find the first hotel search result.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [37]:
# Create dataframe and empty column
hotel_df = pd.DataFrame()
p3 = pd.DataFrame(paramater3)

hotel_df["City"] = p3["City"]
hotel_df["Country"] = p3["Country"]
hotel_df["Lat"] = p3["Lat"]
hotel_df["Lng"] = p3["Lng"]
hotel_df["Hotel Name"] = ''

hotel_df



,City,Country,Lat,Lng,Hotel Name
2,Jamestown,US,42.0970,-79.2353,
11,Thiès,SN,14.8342,-17.1061,
19,Saint George,US,37.1041,-113.5841,
21,Broome,US,42.2506,-75.8330,
24,Tessalit,ML,20.1986,1.0114,
53,Dakar,SN,14.6937,-17.4441,
76,Beckley,US,37.7782,-81.1882,
81,Laguna,US,38.4210,-121.4238,
162,La Reforma,MX,25.0833,-108.0500,
168,Jati,PK,24.3539,68.2671,


In [47]:
# Loop through dataframe and add nearby hotel result

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

paramaters = {"type" : "hotel",
              "keyword" : "hotel",
              "radius" : 5000,
              "key" : g_key}
              
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]             

    paramaters["location"] = f"{lat},{lng}"
    
    print(f"{index}: {city_name}")
    response = requests.get(url, params=paramaters).json()
    results = response['results']
    
    try:
        print(f"{city_name} and {results[0]['name']}")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']   
              
    except (KeyError, IndexError):
        print("skipping")
              
              
        







2: Jamestown
Jamestown and Chautauqua Harbor Hotel
11: Thiès
skipping
19: Saint George
Saint George and Hilton Garden Inn St. George
21: Broome
skipping
24: Tessalit
skipping
53: Dakar
Dakar and Hotel Terrou-Bi
76: Beckley
Beckley and Microtel Inn & Suites by Wyndham Beckley East
81: Laguna
Laguna and Hampton Inn & Suites Sacramento-Elk Grove Laguna I-5
162: La Reforma
La Reforma and Hotel Reforma
168: Jati
Jati and Temra Hotel Jati
182: Jeannette
Jeannette and Holiday Inn Express & Suites Greensburg, an IHG Hotel
185: Arraial do Cabo
Arraial do Cabo and Méditerranée Hotel Arraial do Cabo
194: Mar del Plata
Mar del Plata and Hotel CR Playa Suites
201: Nouakchott
Nouakchott and Mauricenter Hotel
214: Cayenne
Cayenne and Mercure Cayenne Royal Amazonia
272: Elk City
Elk City and Holiday Inn Express & Suites Elk City, an IHG Hotel
296: Taoudenni
skipping
301: Rodolfo Sánchez Taboada
Rodolfo Sánchez Taboada and Hotel Baja
310: Praia
Praia and Hotel Santiago
311: New Castle
New Castle and Ho

In [56]:
# NOTE: Do not change any of the code in this cell
# Analyze the code below for variables but these should be all columns in hotel_df

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]

In [57]:
# Add marker layer ontop of heat map



# Display figure



In [58]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
2,Jamestown,US,42.0970,-79.2353,Chautauqua Harbor Hotel
11,Thiès,SN,14.8342,-17.1061,
19,Saint George,US,37.1041,-113.5841,Hilton Garden Inn St. George
21,Broome,US,42.2506,-75.8330,
24,Tessalit,ML,20.1986,1.0114,
53,Dakar,SN,14.6937,-17.4441,Hotel Terrou-Bi
76,Beckley,US,37.7782,-81.1882,Microtel Inn & Suites by Wyndham Beckley East
81,Laguna,US,38.4210,-121.4238,Hampton Inn & Suites Sacramento-Elk Grove Lagu...
162,La Reforma,MX,25.0833,-108.0500,Hotel Reforma
168,Jati,PK,24.3539,68.2671,Temra Hotel Jati


In [62]:
#plot hotel markers on heat map


hotel_markers = gmaps.marker_layer(hotel_locations, info_box_content = hotel_info)


figure.add_layer(hotel_markers)
plt.savefig("hotelvacationheatmap.png")
figure

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>